# 6.7 GPUs

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def cpu():
    return torch.device('cpu')

def gpu(i=0):
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [3]:
def num_gpus():
    return torch.cuda.device_count()

num_gpus()

8

In [4]:
def try_gpu(i=0):
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(3), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cuda', index=3),
 [device(type='cuda', index=0),
  device(type='cuda', index=1),
  device(type='cuda', index=2),
  device(type='cuda', index=3),
  device(type='cuda', index=4),
  device(type='cuda', index=5),
  device(type='cuda', index=6),
  device(type='cuda', index=7)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1718, 0.3681, 0.5882],
        [0.7543, 0.0795, 0.9209]], device='cuda:1')

In [8]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


In [9]:
Y + Z

tensor([[1.1718, 1.3681, 1.5882],
        [1.7543, 1.0795, 1.9209]], device='cuda:1')

In [10]:
Z.cuda(1) is Z

True

In [11]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

/home/solee/miniconda3/envs/dive2dl/lib/python3.7/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [12]:
net(X)

tensor([[-0.2411],
        [-0.2411]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)

In [16]:
@d2l.add_to_class(d2l.Trainer) #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer) #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer) #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model

AttributeError: module 'd2l.torch' has no attribute 'add_to_class'